In [1]:
%cd ..

d:\workspace\VQA


In [ ]:
import pandas as pd
from PIL import Image
import os
import json
from config import  config
from model.vqa_model import VQAModel
from transformers import AutoTokenizer
import torch

c:\Users\Admin\anaconda3\envs\vqa\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from data_processor import create_dataloder, VQADataset
from torchvision import transforms
df_train = pd.read_csv("./dataset/csv/train_data.csv")
df_val = pd.read_csv("./dataset/csv/dev_data.csv")

criterion = torch.nn.CrossEntropyLoss()
transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])


train_dataset = VQADataset(dataframe=df_train, transform=transforms)
val_dataset = VQADataset(dataframe=df_val, transform=transforms)


train_loader, val_loader = create_dataloder()

data = next(iter(train_loader))
model = VQAModel(text_model=config.text_model, 
                 image_model=config.image_model)

image, question, answer = data

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [6]:
prediction, ans_tokens = model(image, question, answer)
ans_tokens = ans_tokens.long()
print(prediction.shape)
print(ans_tokens.shape)


loss = criterion(prediction.permute(0, 2, 1), ans_tokens)
print(loss)

d:\workspace\VQA\model\vqa_model.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  h_attn = F.softmax(h_attn)


torch.Size([16, 50, 64001])
torch.Size([16, 50])
tensor(11.5250, grad_fn=<NllLoss2DBackward0>)


In [19]:
(torch.argmax(prediction, dim=2) == ans_tokens).sum().item()

0

In [9]:
tokenizer = AutoTokenizer.from_pretrained(config.text_model)

ans_decode = tokenizer.batch_decode(ans_vocab, skip_special_tokens=True)
print(ans_decode)

['họ đang đi mua_sắm trong siêu_thị', 'tỉnh thái_bình', 'có bốn người đang ở trong phòng', 'dùng cho dịch covid-19', 'cổng khu di_tích có ba lối vào', '7 700 đồng một ký', 'trên thành giếng có con_số 1554', 'trên bức tường có treo hai cái đèn_lồng', 'bán hàng', 'vì sếp tôi vừa đẹp_trai vừa tử_tế', 'dự_án khu dân_cư đông bình_dương đang triển_khai', 'bảng thông_báo san_nhượng mặt_bằng', 'một nhân_viên', 'màu đỏ và màu trắng', 'với một người phụ_nữ', 'nước_mắm này có truyền_thống trăm năm']
